# Домашнее задание по курсу анализа данных # 2
Версия от 14.11.2021

Всего 50 баллов = 50 основных + 10 дополнительных

**Оценка**  = (Количество баллов за работу*2 + 20)/10

**Мягкий дедлайн: 06.12.2021  23:59**

**Жесткий дедлайн: 11.12.2021  23:59**

Присылать решение ссылкой на ваш **github** в телеграмм **@Aleron75** или на почту **milenkin.aa@phystech.edu**

**Автор:** Миленькин Александр

# Парсим рейтинг преподавателей с Вики-Физтех

<div class="alert alert-info">
    
Итак, перед вами **VIKI-MIPT** - сборник отзывов о кафедрах и преподавателях МФТИ: ` http://wikimipt.org/wiki/ `

Эта кладезь отзывов не раз давала студентам оценить психологический портрет потенциальных экзаменаторов и преподавателей. 


# Описание задачи: 

**Задача** - собрать данные по всем текущим преподавателям всех `институтских кафедр` МФТИ.

Собранный датасет должен представлять из себя `.csv` файл со следующими полями:

* `full_name` - ФИО преподавателя
* `birth_day` - Дата рождения преподавателя
*  `teach_place` - Кафедра
*  `degree`- Ученая Степень
* `knowledge` - Знания
*  `teaching_skill` - Умение преподавать	
* `commication_skill` - В общении	
* `easy_exam` - «Халявность»	
* `overall_score` - Общая оценка


**Комментарий:** - Знание, умение преподавать, в общение, "Халявность" и Общая оценка можно найти странице каждого преподавателя. 

<div class="alert alert-warning">
    
Финальный балл складывается из трех частей:

* Cобранный датасет (выложить архивом на гитхаб)
* python кода, которым был собран датасет (выложить на гитхаб)
* Окуратный и понятный код с комментариями
    
    
**Дополнительные + 10 баллов можно получить за кластеризацию преподавателей, используя только оценки из собранных данных**

# Возможный алгоритм обхода для сбора данных:

In [ ]:
# 0 - Открываем с помощью Selenium клавную страницу вики-Физтех- http://wikimipt.org/wiki/   
# 1 - Получаем список кафедр с главной страницу вики-Физтех    
# Обходя список кафедр, "проваливаемся" в каждую страницу кафедру кликом по названию кафедры
#     2 - Получаем список преподавателей
#         Обходя список преподавателей "Проваливаемся" на страницу преподавателя по ФИО преподавателя
#             3 - Получаем табличку данных с характеристмиками преподавателями и прочу необходимую информацию. 
#             Записываем в pandas dataframe
# Сохраняем данные в .csv файл

# Вспомогательные элементы для майнинга данных

* Библиотека `Selenium`: https://selenium-python.readthedocs.io/

Полезные методы и рекомендации:
    
* `element = browser.find_elements_by_*что угодно*()` # Поиск элемента на странице
* `element.click()` # Кликнуть по элементу (провалится по ссылке / расскрыть банер)
* `browser.back()` # Вернуться назад 
* `pd.read_html()` # Способ загрузить таблицку из html сразу в Pandas
* Спрашивать в чате или гуглить

In [212]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.by import By



In [ ]:
chrome_options = Options() 
chrome_options.add_argument("--headless")
road_driver = (r'C:\Users\Strong Password\Documents\FILO\МФТИ\АД\my_homework\chromedriver.exe')

driver = webdriver.Chrome(road_driver, options=chrome_options)
driver.get("http://wikimipt.org/wiki/");

In [ ]:
# Отдельно для проверки найдет атрибуты для каждой кафедры

xpath_list_faculty = '//*[@id="mw-content-text"]/div[3]/ul/li/a' # XPATH элемента одной кафедры
faculty = driver.find_elements(By.XPATH, xpath_list_faculty)  # Найдем все элементы каждой кафедры
print("Количество кафедр: ", len(faculty))

url_faculty_list = [el.get_attribute('href') for el in faculty] # Выбираем атрибут URL каждой кафедры
print("\nURL кафедр: \n", url_faculty_list)
faculty_name_list = [el.get_attribute("title") for el in faculty]  # Выбираем атрибут title каждой кафедры
print("\nНазвания кафедр: \n", faculty_name_list)



**Ура, всё работает! Тогда идём дальше**

In [220]:
# Функция парсинга начала таблицы
def start_table_professor(url):
    table = pd.read_html(url)[0] # Читаем таблицу
    driver.implicitly_wait(1)
    notnan_rows = table[0].notnull() # Маска ненулевых значений 
    table = table.loc[notnan_rows, :].values # Выбираем ненулевые значения и делаем в пр
    mask = np.isin(table[:, 0], [ 'Дата рождения', 'Альма-матер', 'Учёная степень'])
    info = {
            'date_of_birth': np.nan,
            'teach_place':   np.nan,
            'degree':        np.nan
            }
    
    alias = {
        'Дата рождения': 'date_of_birth',
        'Альма-матер': 'teach_place',
        'Учёная степень': 'degree'
    }

    for column, value in table[mask, :]:
        if value.endswith(')'): # Нам не нужно сохранять год, в котором окончил вуз преподаватель
            value = value[:-7]
        label = alias[column]
        info[label] = value
        
    return info

In [177]:
# Функция таблицы рейтинга
def rating(driver):
    # Рейтинги всех преподавателей имеют имя класса starrating-avg и идут на странице подряд
    rating = []
    html_rating = driver.find_elements(By.CLASS_NAME, 'starrating-avg')
    driver.implicitly_wait(2)
    for el in html_rating:
        tmp = el.text.strip().split(' ')[0]
        if (tmp and '(' not in tmp):
            number = float(tmp)
        else:
            number = np.nan # заполняем пропуски NaN
    
        rating.append(number)
    rating_by_columns = {
            'knowledge': rating[0],
            'teaching_skill': rating[1],
            'commication_skill': rating[2],
            'easy_exam': rating[3],
            'overall_score': rating[4]}
    return rating_by_columns

In [ ]:
data_for_df = [] # Массив строк для датасета
start = 13
for url_f, name_f in zip(url_faculty_list[start:], faculty_name_list[start:]):
    driver.get(url_f)
    print(name_f)
    driver.implicitly_wait(2)
    if url_f == (url_faculty_list[0] or url_faculty_list[1]):
        xpath_list_professor = '//*[@id="mw-content-text"]/div[3]/ul/li/a' # XPATH элемента одного преподавателя
        professor = driver.find_elements(By.XPATH, xpath_list_professor)
    elif url_f == (url_faculty_list[12]):
        continue
    else:
        prof = driver.find_element(By.CLASS_NAME, "srf-gallery")
        professor = prof.find_elements(By.XPATH, 'ul/li/div/div[2]/p/a')
 
    url_prof_list = [el.get_attribute('href') for el in professor] # Выбираем атрибут URL каждой кафедры
    prof_name_list = [el.get_attribute("title") for el in professor] 
    for url_p, name_p in zip(url_prof_list, prof_name_list):
        driver.get(url_p)
        driver.implicitly_wait(2)
        info_first = start_table_professor(url_p)
        info_rating = rating(driver)
        data_string = {
                        'full_name': name_p,
                        'department': name_f,
                    }
        data_string.update(info_first)
        data_string.update(info_rating)
        #print(data_string)
        data_for_df.append(data_string)

In [ ]:
df = pd.DataFrame(data_for_df)
print(df.shape)
df.to_csv('../Data_hw_2.csv')